# Albedo Radiative Forcing Demo
Interactive-style walkthrough: choose a surface, apply an albedo perturbation, compute top-of-atmosphere (TOA) forcing, and validate against a benchmark sensitivity.

In [4]:
import sys, pathlib
import pandas as pd

# Ensure repo root is on the path when running from notebooks/
repo_root = pathlib.Path.cwd().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from src import albedo, model, validation

# Inspect available surface classes
pd.DataFrame.from_dict(albedo.SURFACE_LIBRARY, orient='index')

,typical,range_min,range_max,note
vegetation,0.17,0.13,0.20,closed canopy forest/grass
desert,0.38,0.30,0.45,bright sand
snow_fresh,0.78,0.70,0.85,fresh dry snow
snow_aged,0.50,0.40,0.60,aging or dusty snow
urban,0.16,0.12,0.20,built environment
cropland,0.20,0.15,0.25,bare soil or sparse crop


## Single-scenario calculation
Edit the parameters below to explore a different surface or perturbation.

In [5]:
# User-editable parameters
surface = 'vegetation'      # pick from the table above
albedo_delta = -0.02         # additive change (negative = darkening)
area_fraction = 0.5          # fraction of Earth affected
anchor = 'typical'           # 'typical', 'min', or 'max' baseline

scenario, forcing_result = model.albedo_pipeline(
    surface_type=surface,
    albedo_delta=albedo_delta,
    anchor=anchor,
    area_fraction=area_fraction,
)
scenario, forcing_result

(Scenario(initial_albedo=0.17, final_albedo=0.15000000000000002, area_fraction=0.5),
 ForcingResult(delta_albedo=-0.01999999999999999, area_fraction=0.5, radiative_forcing_w_m2=3.402499999999998))

In [6]:
# Validate against IPCC-style benchmark sensitivity (-340 W/m^2 per unit Δα, scaled by area)
validation_result = validation.validate_forcing_result(forcing_result)
validation_result

ValidationResult(expected_range_w_m2=(2.7199999999999984, 4.079999999999998), modeled_w_m2=3.402499999999998, within_range=True, notes='Within expected range based on S0/4 scaling.')

## Sensitivity sweep
Explore forcing across multiple perturbations for one surface type.

In [7]:
surface = 'urban'
area_fraction = 0.2
deltas = [-0.05, -0.02, 0.0, 0.02, 0.05]
rows = []
for d in deltas:
    scen, force = model.albedo_pipeline(surface, albedo_delta=d, area_fraction=area_fraction)
    val = validation.validate_forcing_result(force)
    rows.append({
        'delta_alpha': d,
        'forcing_W_m2': force.radiative_forcing_w_m2,
        'within_expected': val.within_range,
        'expected_low': val.expected_range_w_m2[0],
        'expected_high': val.expected_range_w_m2[1],
    })
pd.DataFrame(rows)


,delta_alpha,forcing_W_m2,within_expected,expected_low,expected_high
0,-0.05,3.4025,True,2.720,4.080
1,-0.02,1.3610,True,1.088,1.632
2,0.00,-0.0000,True,-0.000,0.000
3,0.02,-1.3610,True,-1.632,-1.088
4,0.05,-3.4025,True,-4.080,-2.720
